In [ ]:
# Import necessary libraries and packages
import json

In [ ]:
# Load json files
def load_json_file(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

Cost = load_json_file("Cost.json")
G = load_json_file("G.json")
Coord = load_json_file("Coord.json")
Dist = load_json_file("Dist.json")

## Understanding the data 

In [ ]:
# Check type of the loaded data
print(type(Cost))
print(type(G))
print(type(Coord))
print(type(Dist))

In [ ]:
# Check the length of the dictionaries
print(f"Length of Cost dictionary: {len(Cost)}") if isinstance(Cost, (list, dict)) else "Not a list or dict"
print(f"Length of G dictionary: {len(G)}") if isinstance(G, (list, dict)) else "Not a list or dict"
print(f"Length of Coord dictionary: {len(Coord)}") if isinstance(Coord, (list, dict)) else "Not a list or dict"
print(f"Length of Dist dictionary: {len(Dist)}") if isinstance(Dist, (list, dict)) else "Not a list or dict"

### Graph dictionary G
- The graph is given as an adjacency list where the neighbor list of node ‘v’ can be accessed with G[‘v’].


In [ ]:
# Structure of G dictionary
def print_structure(data, iterations=5):
    i = 0
    if isinstance(data, dict):
        for key in data:
            print("Key: ", str(key))
            print_structure(data[key])
            i += 1
            if i >= iterations:
                return
    else:
        print(str(type(data)) + ' ' + str(data))
    
print_structure(G)

In [ ]:
# Example usage of G dictionary
print("Neighbouring nodes of node 1: ", G["1"])

### Node coordination dictionary Coord
- The coordination of a node ‘v’ is a pair (X, Y) which can be accessed with Coord[‘v’]


In [ ]:
print_structure(Coord)

In [ ]:
# Example usage of Coord dictionary
print("Coordinates of node 1: ", Coord["1"])
print("Coordinates of node 2: ", Coord["2"])

### Edge distance dictionary Dist
The distance between a pair of node (v, w) can be accessed with Dist[‘v,w’].

In [ ]:
print_structure(Dist)

In [ ]:
# Example usage of Dist dictionary
print("Distance between nodes 1 and 2: ", Dist["1,2"])

### Edge cost dictionary Cost
The energy cost between a pair of node (v, w) can be accessed with Cost[‘v,w’.]

In [ ]:
print_structure(Cost)

In [ ]:
# Example usage of Cost dictionary
print("Cost of travelling between node 1 and 2: ", Cost["1,2"])

### Combining the graphs together
- except coordinates

In [97]:
combined_graph = {}

for node, neighbors in G.items():
    combined_graph[node] = []
    for neighbor in neighbors:
        edge_key = f"{node},{neighbor}"
        edge_data = {
            "node": neighbor,
            "distance": Dist.get(edge_key, float('inf')), # Using inf as a default if no distance is provided
            "cost": Cost.get(edge_key, float('inf')) # Using inf as a default if no cost is provided
        }
        combined_graph[node].append(edge_data)

print_structure(combined_graph)


Key:  1
<class 'list'> [{'node': '1363', 'distance': 2428, 'cost': 6070}, {'node': '12', 'distance': 1004.7188661511238, 'cost': 2105}, {'node': '2', 'distance': 803, 'cost': 2008}]
Key:  2
<class 'list'> [{'node': '13', 'distance': 704.9198536003934, 'cost': 1478}, {'node': '1', 'distance': 803, 'cost': 2008}, {'node': '48', 'distance': 617, 'cost': 1541}]
Key:  3
<class 'list'> [{'node': '4', 'distance': 158, 'cost': 395}, {'node': '3874', 'distance': 1667, 'cost': 4167}]
Key:  4
<class 'list'> [{'node': '3926', 'distance': 294.7626163542453, 'cost': 627}, {'node': '3', 'distance': 158, 'cost': 395}, {'node': '3937', 'distance': 725.041378129552, 'cost': 1541}]
Key:  5
<class 'list'> [{'node': '6', 'distance': 923.7819006670352, 'cost': 1935}, {'node': '1204', 'distance': 2020.3960007879643, 'cost': 4801}, {'node': '1214', 'distance': 1603.1219541881396, 'cost': 3361}, {'node': '1219', 'distance': 3382, 'cost': 8456}]


In [ ]:
import math

def get_displacement(node_from, node_to):
    global Coord
    node_from_x, node_from_y = Coord[node_from]
    node_to_x, node_to_y = Coord[node_to]
    d = math.sqrt(pow(abs(node_from_x-node_to_x),2) + pow(abs(node_from_y-node_to_y),2))
    return d

print(get_displacement("3","4"))

In [128]:
import heapq

distance_counter = 0
energy_counter = 0

class Node:
    def __init__(self, node, parent=None, distance=0, heuristic=0, energy=0):
        self.node = node
        self.parent = parent
        self.distance = distance
        self.heuristic = heuristic
        self.energy = energy

    def __lt__(self, other):
        return (self.distance + self.heuristic) < (other.distance + other.heuristic)


def astar_search(initial_node, goal_node, heuristic):
    global combined_graph, energy_counter, distance_counter
    open_list = []
    closed_set = set()

    initial_node = Node(initial_node, distance=0,
                        heuristic=heuristic(initial_node, goal_node))
    heapq.heappush(open_list, initial_node)

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.node == goal_node:
            return reconstruct_path(current_node)

        closed_set.add(current_node.node)

        for n in get_neighbours(current_node.node):
            if current_node.energy > 287932:
                continue
            if n in closed_set:
                continue

            e = current_node.energy + int(next(item for item in combined_graph.get(
                current_node.node) if item['node'] == n).get('cost'))
            
            g = current_node.distance + int(next(item for item in combined_graph.get(
                current_node.node) if item['node'] == n).get('distance'))
            
            h = heuristic(n, goal_node)
            n = Node(
                n,
                parent=current_node,
                energy=e,
                distance=g,
                heuristic=h,
            )

            heapq.heappush(open_list, n)
            
    return None  # No path found


def reconstruct_path(node):
    global energy_counter, distance_counter
    path = []
    while node:
        path.append((node.node))
        energy_counter += node.energy
        distance_counter += node.distance
        node = node.parent
    path.reverse()
    return path


# Example usage:
if __name__ == "__main__":

    combined_graph = {}

    for node, neighbors in G.items():
        combined_graph[node] = []
        for neighbor in neighbors:
            edge_key = f"{node},{neighbor}"
            edge_data = {
                "node": neighbor,
                # Using inf as a default if no distance is provided
                "distance": Dist.get(edge_key, float('inf')),
                # Using inf as a default if no cost is provided
                "cost": Cost.get(edge_key, float('inf'))
            }
            combined_graph[node].append(edge_data)

    # Define your initial node, goal node, get_neighbours function, and heuristic function.
    # Example:
    initial_node = "1"
    goal_node = "50"
    combined_graph

    def get_neighbours(node):
        global combined_graph
        neighbours = []
        for n in combined_graph.get(node):
            new_node = n['node']
            neighbours.append(new_node)
        return neighbours

    def heuristic(node_from, node_to):
        global Coord
        node_from_x, node_from_y = Coord[node_from]
        node_to_x, node_to_y = Coord[node_to]
        d = math.sqrt(pow(abs(node_from_x-node_to_x), 2) +
                      pow(abs(node_from_y-node_to_y), 2))
        return d

    path = astar_search(initial_node, goal_node, heuristic)
    if path:
        print("Path found:")
        for node in path:
            if node == "50":
                print(f"{node}")
            else:
                print(f"{node}", end = "->")
        print(f"Total Distance: {distance_counter}")
        print(f"Total Energy: {energy_counter}")

    else:
        print("No path found.")


Path found:
1->1363->1358->1357->1356->1276->1273->1277->1269->1267->1268->1284->1283->1282->1255->1253->1260->1259->1249->1246->963->964->962->1002->952->1000->998->994->995->996->987->988->979->980->969->977->989->990->991->2369->2366->2340->2338->2339->2333->2334->2329->2029->2027->2019->2022->2000->1996->1997->1993->1992->1989->1984->2001->1900->1875->1874->1965->1963->1964->1923->1944->1945->1938->1937->1939->1935->1931->1934->1673->1675->1674->1837->1671->1828->1825->1817->1815->1634->1814->1813->1632->1631->1742->1741->1740->1739->1591->1689->1585->1584->1688->1579->1679->1677->104->5680->5418->5431->5425->5429->5426->5428->5434->5435->5433->5436->5398->5404->5402->5396->5395->5292->5282->5285->5284->5280->50
Total Distance: 145180296
Total Energy: 311246092
